In [1]:
!pip install pyarrow
!pip install fastparquet

In [10]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [11]:
january = pd.read_parquet('yellow_tripdata_2023-01.parquet', engine='fastparquet')
february = pd.read_parquet('yellow_tripdata_2023-02.parquet', engine='fastparquet')

In [12]:
len(january.columns)

19

In [13]:
january.columns

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'airport_fee'],
      dtype='object')

In [14]:
january.sample(5)

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
1591044,2,2023-01-18 07:08:12,2023-01-18 07:17:08,1.0,1.76,1.0,N,162,246,1,10.7,0.0,0.5,3.00,0.0,1.0,17.70,2.5,0.0
871642,2,2023-01-10 21:26:09,2023-01-10 21:33:31,1.0,1.20,1.0,N,162,186,1,9.3,1.0,0.5,2.86,0.0,1.0,17.16,2.5,0.0
1645191,2,2023-01-18 17:27:42,2023-01-18 17:57:05,6.0,4.88,1.0,N,166,170,2,28.2,2.5,0.5,0.00,0.0,1.0,34.70,2.5,0.0
1690723,2,2023-01-19 02:50:25,2023-01-19 02:52:59,1.0,0.39,1.0,N,209,261,1,5.1,1.0,0.5,2.02,0.0,1.0,12.12,2.5,0.0
265161,2,2023-01-04 15:22:33,2023-01-04 15:32:43,1.0,0.91,1.0,N,236,263,1,10.0,0.0,0.5,2.80,0.0,1.0,16.80,2.5,0.0


In [15]:
january['duration'] = (january.tpep_dropoff_datetime - january.tpep_pickup_datetime).dt.total_seconds() / 60

In [16]:
january.duration.std()

np.float64(42.59435124195458)

In [17]:
january.duration.between(1,60).sum()

np.int64(3009173)

In [18]:
january.duration.count()

np.int64(3066766)

In [19]:
january.duration.between(1,60).sum() / january.duration.count() * 100

np.float64(98.1220282212598)

In [20]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import root_mean_squared_error

In [21]:
df = pd.read_parquet('yellow_tripdata_2023-01.parquet', engine='fastparquet')

df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

df = df[(df.duration >= 1) & (df.duration <= 60)]

categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']

df[categorical] = df[categorical].astype(str)

In [22]:
train_dicts = df[categorical + numerical].to_dict(orient='records')

dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

target = 'duration'
y_train = df[target].values

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

root_mean_squared_error(y_train, y_pred)

7.658396898909143

In [25]:
def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)

        df.tpep_dropoff_datetime = pd.to_datetime(df.tpep_dropoff_datetime)
        df.tpep_pickup_datetime = pd.to_datetime(df.tpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)

    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [27]:
df_train = read_dataframe('./data/yellow_tripdata_2023-01.parquet')
df_val = read_dataframe('./data/yellow_tripdata_2023-02.parquet')


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.4 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/home/codespace/.local/lib/python3.12/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/home/codespace/.local/lib/python3.12/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/home/codespace/.local/lib/python3.12/site-packages/ipykernel/kernelapp.py", line 739, in start
    self.io_loop.start()
  File "/home/codes

ImportError: 
A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.4 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.



FileNotFoundError: [Errno 2] No such file or directory: './data/yellow_tripdata_2023-01.parquet'